# How to Use: `calc_mandelbrot_channel`


## Log into OpenBB

In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)


## Collect Data

In [2]:
from humbldata.toolbox.technical.mandelbrot_channel.helpers import add_window_index
from humbldata.toolbox.toolbox_helpers import log_returns

symbols = ["AAPL", "PCT", "NVDA", "SMCI", "TSLA", "AMD", "SPY", "GLD", "BTC-USD", "ETH-USD"]
rr_list = ["MSFT", "AAPL", "AMZN", "META", "GOOGL", "NFLX", "TSLA", "NVDA"]
sector_symbols = [
    "XLF",
    "XLU",
    "XLK",
    "XLE",
    "XLI",
    "XLV",
    "XLY",
    "XLP",
    "XLB",
    "XLRE",
    "XTL"
]


data = (
    obb.equity.price.historical(
        ["AAPL","GOOGL"],
        provider="yfinance",
        start_date="2000-02-08",
        end_date="2024-03-08",
        adjusted=True,
    ).to_polars()
).drop(["dividends", "stock_splits"])

data = log_returns(data, _column_name="close")


## Calculate Mandelbrot Channel


In [6]:
from humbldata.toolbox.technical.mandelbrot_channel.model import calc_mandelbrot_channel


mandelbrot = calc_mandelbrot_channel(
    data,
    window="1y",
    rv_adjustment=True,
    _rv_method="yz",
    _rv_grouped_mean=False,
    _rs_method="RS",
    _live_price=True
).collect()


In [7]:
mandelbrot

date,symbol,bottom_price,recent_price,top_price
datetime[ns],str,f64,f64,f64
2024-03-08 00:00:00,"""AAPL""",170.7113,172.59,185.5596
2024-03-08 00:00:00,"""GOOGL""",136.524,138.525,148.5927
